# SAMWISE Moment Of Inertia Estimation

In [2]:
import numpy as np

In [68]:
class RectangularPrism:
    def __init__(self, 
                 com: np.array,
                 size: np.array,
                 mass: float,
                 R=np.eye(3)) -> None:
        self.com = com   # com in CAD frame [m]
        self.R = R       # rotation matrix mapping local frame to CAD frame (columns are local axes in CAD frame)
        self.size = size # (x, y, z) size in local frame [m]
        self.mass = mass # mass [kg]
        
        assert np.abs(np.linalg.det(R) - 1) < 1e-10 # Rotation matrix should have determinant 1

    @property
    def I_com(self) -> np.array:
        # Moment of inertia about center of mass in local frame [kg m^2]
        # https://dynref.engr.illinois.edu/rem.html
        dx = self.size[0]
        dy = self.size[1]
        dz = self.size[2]

        Ixx = self.mass * (dy**2 + dz**2) / 12
        Iyy = self.mass * (dx**2 + dz**2) / 12
        Izz = self.mass * (dx**2 + dy**2) / 12

        return np.array([
            [Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]
        ])
    
    @property
    def I_com_cad_frame(self) -> np.array:
        # Moment of inertia about the center of mass, in the cad frame [kg m^2]
        # TODO Be 100% sure this works
        return self.R @ self.I_com @ self.R.T

    def I_about_point(self, center: np.array) -> np.array:
        # Return the moment of inertia in the cad frame about a given pivot [kg m^2]
        x, y, z = self.com - center
        I_displacement = self.mass * np.array([
            [y**2 + z**2, -x * y, -x * z],
            [-x * y, x**2 + z**2, y * z],
            [-x * z, -y * z, x**2 + y**2],
        ])
        return self.I_com_cad_frame + I_displacement

In [69]:
def print_inertial_properties(bodies: list[RectangularPrism]):
    print("\n\n----------INERTIAL PROPERTIES------------")
    m_overall = sum([body.mass for body in bodies])
    com_overall = sum([body.com * body.mass for body in bodies]) / m_overall
    print(f"Total mass: {m_overall} [kg]")
    print(f"Com in cad frame: {com_overall} [m]")

    I_overall = sum([body.I_about_point(com_overall) for body in bodies])

    print('')
    print("Moment of intertia about center of mass [kg m^2]")
    print(I_overall)

In [78]:
x = np.sqrt(2) / 2

bodies: list[RectangularPrism] = [
    # TODO
    # To populate on thursday...
    RectangularPrism(
        com=np.array([0, 0, 0]),
        size=np.array([1, 1, 3]),
        mass=1,
        R = np.array([
            [x, 0, x],
            [0, 1, 0],
            [-x, 0, x]
        ])
    ),
]

print_inertial_properties(bodies)




----------INERTIAL PROPERTIES------------
Total mass: 1 [kg]
Com in cad frame: [0. 0. 0.] [m]

Moment of intertia about center of mass [kg m^2]
[[ 0.5         0.         -0.33333333]
 [ 0.          0.83333333  0.        ]
 [-0.33333333  0.          0.5       ]]
